<a href="https://colab.research.google.com/github/IshitaB28/neural_networks/blob/master/movie_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

TensorFlow 2.x selected.


In [2]:
tf.__version__

'2.0.0'

In [3]:
!pip install tensorflow --upgrade

     |████████████████████████████████| 86.3MB 48kB/s 
     |████████████████████████████████| 450kB 51.4MB/s 
     |████████████████████████████████| 3.8MB 67.3MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

In [2]:
tfds.list_builders()

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'bair_robot_pushing_small',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'groove',
 'higgs',
 'horses_or_humans',
 'i_naturalist2017',
 'image_label_folder',
 'imagenet2012',
 'imagenet

In [3]:
(train_data, validation_data), test_data=tfds.load(name='imdb_reviews', split=(tfds.Split.TRAIN.subsplit([6,4]), tfds.Split.TEST), as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [4]:
type(train_data)

tensorflow.python.data.ops.dataset_ops._OptionsDataset

In [0]:
train_examples_batch, train_labels_batch=next(iter(train_data.batch(10)))

In [6]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by

In [7]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [0]:
pretrained_model="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer=hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

In [9]:
hub_layer(train_examples_batch[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 3.9819887, -4.4838037,  5.177359 , -2.3643482, -3.2938678,
        -3.5364532, -2.4786978,  2.5525482,  6.688532 , -2.3076782,
        -1.9807833,  1.1315885, -3.0339816, -0.7604128, -5.743445 ,
         3.4242578,  4.790099 , -4.03061  , -5.992149 , -1.7297493]],
      dtype=float32)>

In [0]:
model=tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(train_data.shuffle(10000).batch(512), epochs=20, validation_data=validation_data.batch(512), verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 197ms/step - loss: 0.7252 - accuracy: 0.4841 - val_loss: 0.6845 - val_accuracy: 0.5617
Epoch 2/20
30/30 [==============================] - 5s 165ms/step - loss: 0.6644 - accuracy: 0.6062 - val_loss: 0.6496 - val_accuracy: 0.6286
Epoch 3/20
30/30 [==============================] - 5s 171ms/step - loss: 0.6344 - accuracy: 0.6493 - val_loss: 0.6223 - val_accuracy: 0.6629
Epoch 4/20
30/30 [==============================] - 5s 165ms/step - loss: 0.6045 - accuracy: 0.6806 - val_loss: 0.5914 - val_accuracy: 0.6917
Epoch 5/20
30/30 [==============================] - 5s 162ms/step - loss: 0.5685 - accuracy: 0.7165 - val_loss: 0.5548 - val_accuracy: 0.7218
Epoch 6/20
30/30 [==============================] - 5s 182ms/step - loss: 0.5255 - accuracy: 0.7477 - val_loss: 0.5123 - val_accuracy: 0.7544
Epoch 7/20
30/30 [==============================] - 5s 179ms/step - loss: 0.4795 - accuracy: 0.7814 - val_loss: 0.4698 - val_accuracy: 0.7847
Epoch 

In [15]:
model.predict(["This movie wasn't as expeted"])

array([[0.2914587]], dtype=float32)

In [16]:
model.predict(["This movie had numerous twists and turns and a lot of thrill and suspense"])

array([[0.94369745]], dtype=float32)